In [8]:
import numpy as np
import pandas as pd

def brownian_black_scholes_data_generator(
    num_samples=10000,
    initial_price_range=(50, 150),
    strike_price_range=(50, 150),
    time_to_maturity_range=(0.01, 2),
    risk_free_rate_range=(0.01, 0.05),
    volatility_range=(0.1, 0.5),
    option_types=("Call", "Put")
):
    """
    Generate synthetic data for Black-Scholes option pricing with Brownian motion.

    Parameters:
        num_samples (int): Number of samples to generate.
        initial_price_range (tuple): Range of initial stock prices (S0).
        strike_price_range (tuple): Range of strike prices (K).
        time_to_maturity_range (tuple): Range of times to maturity (T) in years.
        risk_free_rate_range (tuple): Range of risk-free rates (r).
        volatility_range (tuple): Range of volatilities (σ).
        option_types (tuple): Tuple of option types (e.g., "Call", "Put").

    Returns:
        pd.DataFrame: Synthetic dataset for Black-Scholes calculations.
    """
    np.random.seed(42)  # For reproducibility
    
    # Generate initial parameters
    S0 = np.random.uniform(*initial_price_range, num_samples)  # Initial stock price
    K = np.random.uniform(*strike_price_range, num_samples)  # Strike price
    T = np.random.uniform(*time_to_maturity_range, num_samples)  # Time to maturity
    r = np.random.uniform(*risk_free_rate_range, num_samples)  # Risk-free rate
    sigma = np.random.uniform(*volatility_range, num_samples)  # Volatility
    option_type = np.random.choice(option_types, num_samples)  # Option type
    
    # Create DataFrame
    data = pd.DataFrame({
        "Underlying_Price": S0,
        "Strike_Price": K,
        "Time_to_Maturity": T,
        "Risk_Free_Rate": r,
        "Volatility": sigma,
        "Option_Type": option_type,
    })
    
    return data

def monte_carlo_option_pricing(S, K, T, r, sigma, option_type, num_simulations=10000):
    """
    Calculate option price using Monte Carlo simulation.

    Parameters:
        S (float): Current stock price (Underlying price at time t=0).
        K (float): Strike price.
        T (float): Time to maturity (in years).
        r (float): Risk-free interest rate (annualized).
        sigma (float): Volatility of the stock (annualized).
        option_type (str): Type of the option ('Call' or 'Put').
        num_simulations (int): Number of Monte Carlo paths to simulate.

    Returns:
        float: Monte Carlo price of the option.
    """
    # Simulate price paths at maturity
    np.random.seed(42)  # Reproducibility
    Z = np.random.normal(0, 1, num_simulations)  # Standard normal random variables
    S_T = S * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    
    # Calculate payoffs
    if option_type == "Call":
        payoffs = np.maximum(S_T - K, 0)
    elif option_type == "Put":
        payoffs = np.maximum(K - S_T, 0)
    else:
        raise ValueError("Invalid option type. Choose 'Call' or 'Put'.")
    
    # Discount payoffs to present value
    option_price = np.exp(-r * T) * np.mean(payoffs)
    return option_price

# Generate synthetic data
brownian_bs_data = brownian_black_scholes_data_generator(
    num_samples=10000,
    initial_price_range=(50, 150),
    strike_price_range=(50, 150),
    time_to_maturity_range=(0.01, 1),
    risk_free_rate_range=(0.0192, 0.192),
    volatility_range=(0.1, 2)
)

# Add Monte Carlo true option prices
brownian_bs_data["True_Option_Price"] = brownian_bs_data.apply(
    lambda row: monte_carlo_option_pricing(
        S=row["Underlying_Price"],
        K=row["Strike_Price"],
        T=row["Time_to_Maturity"],
        r=row["Risk_Free_Rate"],
        sigma=row["Volatility"],
        option_type=row["Option_Type"]
    ),
    axis=1
)

# Display the first few rows
brownian_bs_data.head()




,Underlying_Price,Strike_Price,Time_to_Maturity,Risk_Free_Rate,Volatility,Option_Type,True_Option_Price
0,87.454012,87.364082,0.732698,0.129471,0.667933,Call,23.088507
1,145.071431,83.291210,0.192667,0.098566,0.280154,Put,0.000000
2,123.199394,67.615391,0.353173,0.185865,0.340083,Put,0.002909
3,109.865848,110.726667,0.666648,0.057039,0.443275,Put,14.011249
4,65.601864,97.662416,0.487268,0.120782,0.486941,Put,28.718086
